In [392]:

# basic http operations
import requests

# basic computing and data wrangling
import pandas as pd
import numpy as np
import json

# geopandas for advanced geojson handling
import geopandas

# visualizing geo data
import folium

# library to access overpass api in a more convenient way
import overpass

# interactions with the host system 
import os
import time

# pattern matching scala style
# from pampy import match, _

# machine learning
import xgboost as xgb

from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier

In [2]:
overpass_api = overpass.API(timeout=600)


In [3]:

query = """
area["name"="Wien"]["admin_level" ="4"];
relation["boundary"="administrative"]["admin_level"="9"](area);
(._;>;);
out;
"""

response = overpass_api.get(query, responseformat="json")


Since I found no possibility convert the raw json output of overpass to geojson format, here comes a little trick. We need to use a external programm to achieve this. The programm we use is called 'osmtogeojson' and is written to run on the node.js environment. 

If you want to try the steps below, you need install node.js and then install 'osmtogeojson' by running 'npm install -g osmtogeojson' in your console (npm is the node.js equivalent of pip).

We save the downloaded raw json to the file system und then run osmtogeojson and convert it to a geojson file.


In [4]:
with open("vienna_bounds_raw.json", "w") as file:
    file.write(json.dumps(response))
    
os.system('osmtogeojson vienna_bounds_raw.json > vienna_bounds.geojson')


0

We can now import the generated geojson file using the library geopandas, a pandas derivative that is mainly made for processing geo data like we saved in our geojson. 

Since the geojson file we downloaded still contains data we don't need and which spoils our our visualizations later on, we filter the imported file and save it to the file system again so we can use it with folium later. 


In [5]:
gdf_vienna_bounds = (geopandas.read_file('vienna_bounds.geojson')
                    .loc[:, ['id','name', 'geometry', 'ref']]
                    .query("id.str.startswith('relation')", engine="python")
            )

gdf_vienna_bounds.to_file("vienna_bounds.geojson", driver='GeoJSON')


Lets check our DataFrame containing the data about Viennas districts.

In [6]:
gdf_vienna_bounds.head()


,id,name,geometry,ref
0,relation/1990590,Alsergrund,"POLYGON ((16.3567669 48.2361278, 16.3567424 48...",9
1,relation/1990591,Hietzing,"POLYGON ((16.2584479 48.1972588, 16.2580803 48...",13
2,relation/1990592,Innere Stadt,"POLYGON ((16.3659409 48.1996821, 16.3659676 48...",1
3,relation/1990593,Josefstadt,"POLYGON ((16.3392972 48.2122485, 16.3392509 48...",8
4,relation/1990594,Leopoldstadt,"POLYGON ((16.497085 48.1662737, 16.4984617 48....",2


To make shure we downloaded the correct data, we can visualize with the help of the folium map visualization library.


In [7]:
vienna_map = folium.Map(location=(48.2204,16.3367), zoom_start=11) 

folium.Choropleth(
    geo_data="vienna_bounds.geojson"
).add_to(vienna_map)

                  
vienna_map


## Calculate centroids

As we need a reference point for our foursquare API, we will simply get the centroids of the single polygons representing Viennas districts. As we are using geopandas, which uses shapely under the hood, this is very easy to accomplish.

In [8]:
gdf_vienna = (gdf_vienna_bounds
                .assign(center_lat=lambda x: x['geometry'].centroid.y)
                .assign(center_lon=lambda x: x['geometry'].centroid.x)
             )

Lets display these centroids on the map to check if they look realistic.

In [9]:
for i, district in gdf_vienna.iterrows():
    

    folium.Marker(
        (district['center_lat'], district['center_lon'])
        
    ).add_to(vienna_map)
    
vienna_map

Looks good. Lets continue with retrieving data from foursquare.

## Retrieve trending venues from foursquare

Set foursquare credentials. (Since its a free account I don't really care if they are available on github)

In [10]:
CLIENT_ID = 'TLLSXQMPOKLY0NLU0ZRA4MUW45XBYO5G0P5B5VFW0ZRJMY2U'
CLIENT_SECRET = 'QH2K33NY2RZQPSHRBYGLNMFDYYEPLUTOA44IND2NCDEQVXDD'
VERSION = '20190706'

In [11]:
def get_trending_venues(lat, lon, radius, limit):

    url = "https://api.foursquare.com/v2/venues/explore"
    
    
    
    params = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "v": VERSION,
        "ll": "{lat},{lon}".format(lat=lat, lon=lon),
        "radius": radius,
        "limit": limit,
        "section": 'trending',
        "query": 'food'
    }

    return requests.get(url, params).json()['response']['groups'][0]['items']

We define a function that takes all venues we got for a district and counts them grouped by their primary category. 

In [12]:
def group_venues(venues, d_ref, d_name):
    
    list_venues = (venue['venue']['categories'][0]['name'] for venue in venues) 
    
    df_venues = pd.DataFrame(list_venues, columns=["name"])
    
    
    return (df_venues
             
             .groupby('name')['name']
                .count()
                .to_frame()
             .transpose()
             .reset_index()
             .assign(d_ref=d_ref)
             .assign(d_name=d_name)
             .drop(columns=["index"])
           )

Now we use our functions we defined before to retrieve all venues for all districts and build a list of dataframes. This list can be concatinated later on to receive one 

In [14]:
trending_venues = []

for i, district in gdf_vienna.iterrows():
    
    venues = get_trending_venues(district['center_lat'], district['center_lon'], 2000, 110)
    
    trending_venues.append(group_venues(venues, district['ref'], district['name']))

In [15]:
df_trending = (pd.concat(trending_venues, sort=False)
               .fillna(0)
              )

Lets drop all venue categories that don't contain at least 10 entries as they are to specific and it will be impossible in a future implementation to ask the user for his preferences, given so many categories.

In [16]:
df_trending_feature_sel = df_trending.drop(columns=["d_ref", "d_name"]).apply(pd.to_numeric)

is_small = (df_trending_feature_sel.sum() < 10)
col_del = is_small[is_small].index

df_vienna_features = (df_trending
                          .drop(columns=col_del)
                          .reset_index()
                          .drop(columns=["index"])
                     )

Just a short peek on what we've got so far:

In [17]:
df_vienna_features.head()

,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Café,Chinese Restaurant,Falafel Restaurant,Gastropub,...,Thai Restaurant,Trattoria/Osteria,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,d_ref,d_name,Bistro,Hot Dog Joint,Steakhouse,Fast Food Restaurant
0,5.0,6,1.0,6.0,1.0,3.0,13.0,2.0,4.0,5.0,...,2.0,2.0,1.0,2.0,9,Alsergrund,0.0,0.0,0.0,0.0
1,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,13,Hietzing,0.0,0.0,0.0,0.0
2,4.0,14,1.0,3.0,1.0,3.0,8.0,1.0,1.0,3.0,...,2.0,2.0,2.0,0.0,1,Innere Stadt,1.0,1.0,3.0,0.0
3,5.0,9,0.0,5.0,1.0,3.0,13.0,2.0,1.0,3.0,...,1.0,1.0,1.0,2.0,8,Josefstadt,1.0,1.0,1.0,0.0
4,2.0,3,1.0,8.0,0.0,1.0,10.0,1.0,0.0,3.0,...,0.0,1.0,0.0,0.0,2,Leopoldstadt,1.0,2.0,0.0,2.0


## Load Open Street Maps amenities

In [30]:
def retrieve_amenities(relation_id):
    
    area_id = relation_id + 3600000000 

    query = """
area({area});
way["leisure" = "park"](area);
out center;
 
area({area});
node["shop" = "supermarket"](area);
out; 

area({area});
node["amenity" = "university"](area);
out;

area({area});
node["amenity" = "restaurant"](area);
out;

area({area});
node["amenity" = "doctors"](area);
out;

area({area});
node["amenity" = "waste_basket"](area);
out;
    """.format(area=area_id)
    
    overpass_api.endpoint

    response = overpass_api.get(query, responseformat="json")
    
    return response

In [89]:
def group_places(places, d_name, d_ref, d_area) -> pd.DataFrame:
    

    df_places = pd.io.json.json_normalize(places['elements'])
    
    df_places['category'] = df_places['tags.amenity'].fillna(df_places['tags.leisure'])
    df_places['category'] = df_places['category'].fillna(df_places['tags.shop'])
    
    return  (df_places
                      .groupby('category')['id']
                        .count()
                        .to_frame()
                     .transpose()
                     .reset_index()
                     .assign(index=lambda x: x.index.astype(int),
                         d_area=d_area,
                         d_ref=d_ref,
                         d_name=d_name)
           )
    
    
   


Download all places specified in function retrieve_amenities. 
This will take a while (about 1h) since Overpass API has restrictions on multiple access. 

In [417]:
places = list()


for i, row in gdf_vienna.iterrows():

    
    
    area = row['geometry'].area
    
    
    # Wait 300s if we get any exception and retry. 
    # We don't specify exception type because there are many that lead to a retry
    while True:
        try:
            places_raw = retrieve_amenities(int(row['id'].split('/')[1]))
        
        
        except:
            
            time.sleep(300)
            continue
        
        break
    
    places.append(group_places(places_raw, row['name'], row['ref'], area))
    
    


Conatinate the retrieved singel lined dataframes to a single dataframe, containing all our districts

In [420]:
df_places = (pd.concat(places, sort=False)
                   .reset_index()
                   .drop(columns=['index', 'level_0', 'atm', 'cafe', 'fuel'])
                   .fillna(0)                   
                  )

Save the data we've gathered to .csv so we don't need to wait for an hour next time we need it.

In [422]:
df_places.to_csv('vienna_osm_places.csv')

## Load average flat rental prices

Its not very useful if users get recommendations for districts that fit their preferences well, but that are not affordable for them. 

In [ ]:
df_prices = (pd
             .read_csv("rental_prices_vienna.csv", sep=";")
             .assign(ref = lambda x: x['Bezirk'].str.extract('(\d*)'),
                     district = lambda x: x['Bezirk'].str.extract('([^\d.,]+)'))
             .drop(columns=["Bezirk", "≤50m²", "51-80m²", "81-129m²", ">130m²"])
            )

In [ ]:
df_prices

## Train a XGBClassifier on our dataset 

In [247]:
scaler = preprocessing.MinMaxScaler()




df_X = df_vienna_features.drop(columns=['d_ref', 'd_name'])

X = scaler.fit_transform(df_X.values)

In [353]:
lb = preprocessing.LabelBinarizer()

y = lb.fit_transform(df_vienna_features['d_ref'])

In [380]:

clf = OneVsRestClassifier(xgb.XGBClassifier(objective='reg:squarederror', max_depth=6))



clf.fit(X, y)

OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
          n_jobs=None)

In [475]:
test = clf.predict_proba(X)[0]

In [476]:
test

array([ 2.1606684e-05,  1.3235211e-04, -7.3790550e-05,  2.1606684e-05,
        3.9041042e-05,  1.5619397e-04,  2.1606684e-05,  2.1606684e-05,
        6.5699220e-04,  2.1606684e-05,  4.0590763e-05,  2.1606684e-05,
        2.1606684e-05,  2.8124452e-04,  3.6638975e-04,  3.1822920e-04,
        2.1606684e-05,  2.1606684e-05,  2.1606684e-05, -2.7954578e-05,
        2.1606684e-05,  6.8104267e-04,  9.9703985e-01], dtype=float32)

In [477]:
test.max()

0.99703985

In [478]:
weighted_out = []

for i in range(len(test)-1):
    
    label_bin = np.zeros(test.shape)
   
    label_bin[test.argmax()] = 1
    weight = test.max()
    
    label_bin = np.array([label_bin])
    label = lb.inverse_transform(label_bin)
    
    weighted_out.append(pd.DataFrame([[label[0], weight]]))
    
    test[test.argmax()] = 0
    

    
    

In [479]:
pd.concat(weighted_out)

,0,1
0,9,0.997040
0,8,0.000681
0,17,0.000657
0,22,0.000366
0,23,0.000318
0,21,0.000281
0,14,0.000156
0,10,0.000132
0,19,0.000041
0,13,0.000039
